In [1]:
# !pip install torch
# !pip install torchvision

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class FCNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
# Define model
class SiLUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.SiLU(),
            nn.Linear(512, 512),
            nn.SiLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

fcnet = FCNet().to(device)
silunet = SiLUNet().to(device)
print(fcnet)
print(silunet)

Using mps device
FCNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
SiLUNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): SiLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcnet.parameters(), lr=1e-3)

In [13]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, fcnet, loss_fn, optimizer)
    test(test_dataloader, fcnet, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.180880  [   64/60000]
loss: 0.283697  [ 6464/60000]
loss: 0.204245  [12864/60000]
loss: 0.296938  [19264/60000]
loss: 0.274478  [25664/60000]
loss: 0.363654  [32064/60000]
loss: 0.257079  [38464/60000]
loss: 0.346567  [44864/60000]
loss: 0.336607  [51264/60000]
loss: 0.256889  [57664/60000]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.363947 

Epoch 2
-------------------------------
loss: 0.180880  [   64/60000]
loss: 0.283697  [ 6464/60000]
loss: 0.204245  [12864/60000]
loss: 0.296938  [19264/60000]
loss: 0.274478  [25664/60000]
loss: 0.363654  [32064/60000]
loss: 0.257079  [38464/60000]
loss: 0.346567  [44864/60000]
loss: 0.336607  [51264/60000]
loss: 0.256889  [57664/60000]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.363947 

Epoch 3
-------------------------------
loss: 0.180880  [   64/60000]
loss: 0.283697  [ 6464/60000]
loss: 0.204245  [12864/60000]
loss: 0.296938  [19264/60000]
loss: 0.274478  [25664/60000]
loss: 0.363654  [32064/600

In [14]:
optimizer = torch.optim.Adam(silunet.parameters())

In [15]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, silunet, loss_fn, optimizer)
    test(test_dataloader, silunet, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.176219  [   64/60000]
loss: 0.169617  [ 6464/60000]
loss: 0.241484  [12864/60000]
loss: 0.175193  [19264/60000]
loss: 0.298829  [25664/60000]
loss: 0.264888  [32064/60000]
loss: 0.175309  [38464/60000]
loss: 0.241163  [44864/60000]
loss: 0.266719  [51264/60000]
loss: 0.319686  [57664/60000]
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.351822 

Epoch 2
-------------------------------
loss: 0.127454  [   64/60000]
loss: 0.143113  [ 6464/60000]
loss: 0.227459  [12864/60000]
loss: 0.169539  [19264/60000]
loss: 0.322754  [25664/60000]
loss: 0.218094  [32064/60000]
loss: 0.160329  [38464/60000]
loss: 0.228695  [44864/60000]
loss: 0.240226  [51264/60000]
loss: 0.308378  [57664/60000]
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.367984 

Epoch 3
-------------------------------
loss: 0.142044  [   64/60000]
loss: 0.149387  [ 6464/60000]
loss: 0.201129  [12864/60000]
loss: 0.167811  [19264/60000]
loss: 0.315297  [25664/60000]
loss: 0.226439  [32064/600